<a href="https://colab.research.google.com/github/SankethGB/Market-Basket/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required Libraries.

Kindly ensure that pytorch has been installed prioirly

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable
from mlxtend.frequent_patterns import apriori, association_rules

Import the cleaned dataset


In [6]:
dataset = pd.read_csv("/content/drive/MyDrive/outliers/Cleaned_Data.csv")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
dataset.head()

,Unnamed: 0,Unnamed: 0.1,Doc. Date,Material,Order qty,Ship-to nu,PCS delivered,HL delivered,delivery_days,delivery_flag,lead_time_creation_vs_rdd_flag,MACO/HL,Groupement,Postal Code,Street,Sous groupement,M2_Territory_ID,M1_Territory_ID,Dépt,Brand,Subrand,SEGMENTS : Pils / Spécialités / Superspécialités/Bouteille Young adult,Container Type,Container Size,Variétés,Segment LE,Latitude,Longitude,Degre Alc,TTC,Brut + TE,Net + TE - Hors majoration de rompu de palette,DA
0,0,0,2021-01-11,10946,5.0,29606863,350.0,21.0,7,1,1,150.689825,FRANCE BOISSONS,59160,RUE DU CHEMIN SAINT MARTIN,France BOISSONS,FR01_ON_1000,FR01_ON_1004,59.0,LEFFE,LEFFE BLONDE,Spécialités,PERFECT DRAFT,"6,000 L",Blonde,PREMIUM,50.6455,2.9619,6.6,18.137448,15.582,15.114540,0.0
1,1,1,2021-01-11,19898,2.0,29606863,140.0,8.4,7,1,1,141.207904,FRANCE BOISSONS,59160,RUE DU CHEMIN SAINT MARTIN,France BOISSONS,FR01_ON_1000,FR01_ON_1004,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6455,2.9619,5.0,17.396467,14.688,14.497056,0.0
2,2,2,2021-01-11,80176,4.0,29606863,280.0,16.8,7,1,1,92.872307,FRANCE BOISSONS,59160,RUE DU CHEMIN SAINT MARTIN,France BOISSONS,FR01_ON_1000,FR01_ON_1004,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6455,2.9619,0.0,12.279859,10.368,10.233216,0.0
3,3,3,2021-01-11,66989,1.0,29606863,70.0,4.2,7,1,1,195.334035,FRANCE BOISSONS,59160,RUE DU CHEMIN SAINT MARTIN,France BOISSONS,FR01_ON_1000,FR01_ON_1004,59.0,TRIPLE KARMELIET,TRIPLE KARMELIET,Craft,PERFECT DRAFT,"6,000 L",-,SUPER PREMIUM,50.6455,2.9619,8.0,21.996000,18.330,18.330000,0.0
4,4,4,2021-01-08,11766,70.0,29378784,70.0,4.2,6,1,1,142.902378,INDEPENDANTS,62570,PLACE JEAN JAURES,ALLAN,FR01_ON_2000,FR01_ON_2006,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.6978,2.2423,5.2,17.359200,14.466,14.466000,0.0


List out all unique values of Groupement,material IDs and wholesaler IDs  in dataset




In [30]:
group1 =dataset.iloc[:, 12].unique()
setter1 = set(group1)
groups = (list(setter1))
materials1 =dataset.iloc[:, 3].values
setter = set(materials1)
materials = (list(setter))
buyers1 = dataset.iloc[:, 5].values
settt = set(buyers1)
buyers = (list(settt))

***One with the apriori algorithm***





*   Reducing the entire dataset selecting the columns which are useful for the process input
*   Reassuring that the necessary fields are all non-null
*   Creating a column called invoice which represents the ID for an invoice which includes all the purchases of a particular person in a single day.






In [31]:
data = dataset[["Doc. Date", "Material","Ship-to nu", "Groupement", "Order qty"]]

data.dropna(axis=0, subset=["Groupement"], inplace=True)
data.dropna(axis=0, subset=["Material"], inplace=True)
data.dropna(axis=0, subset=["Ship-to nu"], inplace=True)

data["invoice"] = data["Doc. Date"].astype(str)+'m'+data["Ship-to nu"].astype(str)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [55]:
print(buyers)

[29804035, 29379593, 29379089, 29379604, 29379608, 29379609, 29379610, 29379611, 29379612, 29379613, 29379101, 29379615, 29607967, 29379618, 29379619, 29379620, 29379621, 29379622, 29379623, 29379625, 29379626, 29379627, 29379116, 29379628, 29379632, 29379633, 29379635, 29379639, 29379641, 29379642, 29986364, 29724733, 29379646, 29750335, 29379650, 29379651, 29839429, 29379657, 29379658, 29379661, 29379663, 29379664, 29801553, 29379666, 29379667, 29379668, 29379671, 29379672, 29379673, 29379676, 29379677, 40016990, 29379678, 29379680, 29379681, 29379682, 29379684, 29564005, 29840997, 29379687, 29563495, 29379686, 29379690, 29379692, 29379694, 29379695, 29379696, 29430384, 29379698, 29446255, 29379701, 29379702, 29379708, 29610633, 29799562, 29983887, 29643421, 29780131, 29379236, 29563563, 29793472, 40007881, 29379283, 29988565, 29727962, 29378784, 29378785, 40002273, 29379815, 29379818, 29379819, 29379820, 29635311, 29379824, 29379826, 29379827, 29379828, 29379829, 29379830, 29379831,

A function that hot encodes all values to 0 and 1 defined to deduce if a material is being included in that particular buy (order quantity>0) or not. (no such order or order quatity=0) 

In [33]:
 def hot_encode(x):
        if(x <= 0):
            return 0
        if(x >= 1):
            return 1

Declaring a list of dictionaries. Each dictionary represents a particular groupement containing lists of recommendations for a particular material ID. It was observed that the patterns gained more support after the groupement filter. It is obvious that customers belonging to different groupements have different buying patterns. Without the filter the patterns had reduced amount of confidence and support.

In [34]:
dictlist = [dict() for x in range(5)]

Apriori algorithm steps:


*   Creating a 2-D matrix called basket with the columns representing unique invoices and rows representing unique material IDs. For each material ID, the value of a cell is one if it was present during the partcular order, else zero.
*   Apriori function takes up all frequently found items and filters it with min_support value of 0.02.
*   Further association rules are established with a minimum confidence of 0.1 just to ensure all patterns are observed.
*   Of all the patterns the patterns including a single antecedant and a single consequent are filtered since our recommendations would be one-one. Rest patterns can be used for **bundling** and can easily extracted using consequent and antecedent length.
* Further the sorting is done so that patterns/rules with high confidence,lift and support metrices are obtained.



In [41]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
for i in range(5):
   
    basket = (data[data['Groupement'] == groups[i]]
                .groupby(["invoice", 'Material'])['Order qty']
                .sum().unstack().reset_index().fillna(0)
                .set_index('invoice'))
    basket_encoded = basket.applymap(hot_encode)
    frq_items = apriori(basket_encoded, min_support=0.04, use_colnames=True)
    rules = association_rules(frq_items, metric="confidence", min_threshold=0.1)
    rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])
    rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
    rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))
    rule1 = rules[(rules['antecedent_len']+rules['consequent_len'] == 2)]
    rule1 = rule1.sort_values(['confidence', 'lift', 'support'], ascending=[False, False, False])
    rule = rule1.iloc[:, :].values
    from collections import defaultdict
    dictt = defaultdict(list)
    for z in rule:
      for j in z[0]:
          for k in z[1]:
              dictt[j].append([z[5], k])
    dictlist[i] = dictt
  
    
                                                                           

In [43]:
dictlist[0][3338]

[[0.5846153846153846, 59848],
 [0.5538461538461539, 11574],
 [0.49230769230769234, 9974],
 [0.47692307692307695, 71317],
 [0.4615384615384615, 3337],
 [0.4615384615384615, 3410],
 [0.4615384615384615, 6013],
 [0.4461538461538462, 10946],
 [0.4307692307692308, 10957],
 [0.4153846153846154, 3372]]

It is observed that not all material IDs are being part of market patterns. Such cases can be tackled by recommending materials which are similar. This can be deduced using cosine similarity between the materials


***One where the Hectolitres get predicted through auto-encoder***

Preprocessing: 
1. Finding average Hectolitres of each material ordered by each Wholesaler 
2. Scaling them using minmax technique column-wise
3. Pytorch Models accept Float tensors only.So converting the numpy array into a tesor



In [56]:
data =dataset.iloc[:, :].values
x = np.zeros((179, 163))
y = np.zeros((179, 163))
for boom in data:
        if(boom[3] > 0):
            x[buyers.index(boom[5])][materials.index(boom[3])] += boom[7]
            y[buyers.index(boom[5])][materials.index(boom[3])] += 1
for i in range(179):
    for j in range(163):
        if y[i][j] > 0:
            x[i][j] /= y[i][j]
maxes = np.amax(x, axis=1)
mines = np.amin(x, axis=1)
for i in range(179):
    for j in range(163):
        x[i][j] -= mines[i]
        x[i][j] /= (maxes[i]-mines[i]) 
xsc = torch.FloatTensor(x)        

Model:
* The autoencoder tries to generate from the reduced encoding a representation as close as possible to its original input.
* Performing the copying task perfectly would simply duplicate the signal, and this is why autoencoders usually are restricted in ways that force them to reconstruct the input approximately(here number of hidden nodes are less than the input size preventing direct duplication) ,preserving only the most relevant aspects of the data in the copy.
* Here it tries to copy the patterns in delivered amounts considering the average HLDelivered of a material purchased by a wholsaler in the past. 
1. Creating a class to build a stacked auto-encoder.
2. Initializing three an input layer, two hidden layers and an output layer with the mentioned number of nodes.
3. Declaring sigmoid as the activation function.
4. The forward function involves passing the tensor x through the layers activated by sigmoid function. (Note that fc4 is the output layer and thus will work with linear activation). 


In [57]:
class SAE(nn.Module):
        def __init__(self):
            super(SAE, self).__init__()
            self.fc1 = nn.Linear(163, 25)
            self.fc2 = nn.Linear(25, 12)
            self.fc3 = nn.Linear(12, 30)
            self.fc4 = nn.Linear(30, 163)
            self.activation = nn.Sigmoid()

        def forward(self, x):
            x = self.activation(self.fc1(x))
            x = self.activation(self.fc2(x))
            x = self.activation(self.fc3(x))
            x = self.fc4(x)
            return x

Declaring training parameters: 
* Mean squared error as the metrics for training.
* Learning rate of 0.06 and weight decay of 0.5
*   Number of epochs in training = 50



In [58]:
sae = SAE()
criterion = nn.MSELoss()
optimiser = optim.RMSprop(sae.parameters(), lr=0.06, weight_decay=0.5)
nb_epochs = 50

Training Step:

In [59]:
for epochs in range(1,nb_epochs+1):
    training_loss=0
    s=0.
    for users in range(178):
        xx=Variable(xsc[users]).unsqueeze(0)
        target=xx.clone()
        if torch.sum(target.data > 0) > 0:
            output=sae(xx)
            target.requires_grad=False
            output[target ==0] = 0
            loss=criterion(output,target)
            mean_corrector=(179)/ float(torch.sum(target.data>0)+1e-10)
            loss.backward()
            training_loss+=np.sqrt(loss.data * mean_corrector)
            s+=1.  
            optimiser.step()
    print("Epoch:"+str(epochs)+" Loss: "+str(training_loss/s))

Epoch:1 Loss: tensor(0.2746)
Epoch:2 Loss: tensor(0.2616)
Epoch:3 Loss: tensor(0.2079)
Epoch:4 Loss: tensor(0.1938)
Epoch:5 Loss: tensor(0.1878)
Epoch:6 Loss: tensor(0.1842)
Epoch:7 Loss: tensor(0.1820)
Epoch:8 Loss: tensor(0.1803)
Epoch:9 Loss: tensor(0.1792)
Epoch:10 Loss: tensor(0.1781)
Epoch:11 Loss: tensor(0.1776)
Epoch:12 Loss: tensor(0.1770)
Epoch:13 Loss: tensor(0.1764)
Epoch:14 Loss: tensor(0.1758)
Epoch:15 Loss: tensor(0.1757)
Epoch:16 Loss: tensor(0.1752)
Epoch:17 Loss: tensor(0.1750)
Epoch:18 Loss: tensor(0.1745)
Epoch:19 Loss: tensor(0.1746)
Epoch:20 Loss: tensor(0.1740)
Epoch:21 Loss: tensor(0.1742)
Epoch:22 Loss: tensor(0.1736)
Epoch:23 Loss: tensor(0.1738)
Epoch:24 Loss: tensor(0.1734)
Epoch:25 Loss: tensor(0.1735)
Epoch:26 Loss: tensor(0.1732)
Epoch:27 Loss: tensor(0.1733)
Epoch:28 Loss: tensor(0.1730)
Epoch:29 Loss: tensor(0.1731)
Epoch:30 Loss: tensor(0.1728)
Epoch:31 Loss: tensor(0.1729)
Epoch:32 Loss: tensor(0.1726)
Epoch:33 Loss: tensor(0.1728)
Epoch:34 Loss: tens

Output: Converting the output tensor to list and reversing the scaling operation

In [60]:
output = sae(xsc)
ans = output.tolist()
for i in range(179):
    for j in range(163):
        ans[i][j] *= (maxes[i]-mines[i])
        ans[i][j] += mines[i]
        if(ans[i][j]<0):
            ans[i][j]=0.00001

***One with the Restricted Boltzmann Machine***

Preprocessing:


1.   Creating a numpy array of total materials*total wholesalers.Each entry is one if the corresponding material has been bought by the wholesaler in the past.(If we had data about the rating or review recieved from the wholesaler, the array would then depict the liking of materials.)
2. Converting the array into float tensor which is the desired input to the model



In [62]:
x = np.zeros((179, 179))
for boom in data:
      if(boom[4] > 0 and boom[7] > 0):
            x[buyers.index(boom[5])][materials.index(boom[3])] = 1
xsc = torch.FloatTensor(x)

Model:
* A restricted Boltzmann machine (RBM) is a generative stochastic artificial neural network that can learn a probability distribution over its set of inputs.
* The standard type of RBM has binary-valued (Boolean) hidden and visible units, and consists of a matrix of weights W of size m*n. 
* Each weight element (w[i][j])  of the matrix is associated with the connection between the visible (input) unit v[i] and the hidden unit h[j]. In addition, there are bias weights (offsets) a[i] for v[i]  and b[j] for h[j]. 


1.   init function initializes weights and biases randomly 
2.   sample_h produces probabilities (ph_given_v) of hidden nodes h[j] given the state of visible nodes. It also outputs bernoulli samples corresponding to the calculated probabilities.
3.  sample_v performs the same functions as sample_h just with hidden nodes as input and visible nodes in the output end.
4. Train function uses the principle of contrastive divergence and aligns the weights and biases to minimize the energy of the state.



In [63]:
class RBM():
                    def __init__(self, nv, nh):
                        self.W = torch.randn(nh, nv)
                        self.A = torch.randn(1, nh)
                        self.B = torch.randn(1, nv)

                    def sample_h(self, x):
                        wx = torch.mm(x, self.W.t())
                        activation = wx + self.A.expand_as(wx)
                        p_h_given_v = torch.sigmoid(activation)
                        return p_h_given_v, torch.bernoulli(p_h_given_v)

                    def sample_v(self, y):
                        wy = torch.mm(y, self.W)
                        activation = wy + self.B.expand_as(wy)
                        p_v_given_h = torch.sigmoid(activation)
                        return p_v_given_h, torch.bernoulli(p_v_given_h)

                    def train(self, v0, vk, ph0, phk):
                        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
                        self.B += torch.sum((v0-vk), 0)
                        self.A += torch.sum((ph0-phk), 0)

Parameters for training:


*   Number of columns and rows are both less than 200 and hence a bath size of 1 is sufficient and as expected gave better results.
*   As per previous RBM users considering number of hidden nodes to be nearly equal to half of the number of visible nodes gives higher accuracy. So nh is chosen to be 90 and also obtained better results.




In [64]:
batch_size = 1
rbm = RBM(179, 90)
nb_epochs = 15

Training:

In [65]:
for i in range(1, nb_epochs+1):
        train_loss = 0
        s = 0.
        for id_user in range(1, 179-batch_size, batch_size):
            vk = xsc[id_user:id_user+batch_size]
            v0 = xsc[id_user:id_user+batch_size]
            ph0, _ = rbm.sample_h(v0)
            for k in range(10):
                _, hk = rbm.sample_h(vk)
                _, vk = rbm.sample_v(hk)
                vk[v0 < 0] = v0[v0 < 0]
            phk, _ = rbm.sample_h(vk)
            rbm.train(v0, vk, ph0, phk)
            train_loss += torch.mean(torch.abs(v0[v0 >= 0]-vk[v0 >= 0]))
            s += 1.
        print("epoch: "+str(i)+" loss: "+str(train_loss/s))   

epoch: 1 loss: tensor(0.2134)
epoch: 2 loss: tensor(0.2011)
epoch: 3 loss: tensor(0.1957)
epoch: 4 loss: tensor(0.1919)
epoch: 5 loss: tensor(0.1882)
epoch: 6 loss: tensor(0.1857)
epoch: 7 loss: tensor(0.1800)
epoch: 8 loss: tensor(0.1828)
epoch: 9 loss: tensor(0.1799)
epoch: 10 loss: tensor(0.1773)
epoch: 11 loss: tensor(0.1769)
epoch: 12 loss: tensor(0.1736)
epoch: 13 loss: tensor(0.1793)
epoch: 14 loss: tensor(0.1782)
epoch: 15 loss: tensor(0.1773)


Output and Metrices

In [66]:
from sklearn.metrics import confusion_matrix
ans=[]
avg_rek=0
avg_prk=0
conmatr=[[0,0],[0,0]]
for id_user in range(179):
    v=xsc[id_user:id_user+1]
    _,h=rbm.sample_h(v)
    pv,v1=rbm.sample_v(h)
    pvl=pv.tolist()
    pvn=np.array(pvl)
    indones=pvn[0].argsort()[-25:][::-1]
    rel=0
    for iii in indones:
        if x[id_user][iii]>0 and v1[0][iii]==1:
            rel=rel+1
    conmatr+=confusion_matrix(v[0].tolist(),v1[0].tolist())   
    avg_rek+=rel/float(torch.sum(v))
    avg_prk+=rel/25
    
    ans.append(pvl)
conmatr
avg_rek/=179
avg_prk/=179
print(conmatr,avg_rek,avg_prk)    

[[21717  2214]
 [ 3062  5048]] 0.4781714596353773 0.7405586592178772
